In [2]:
import numpy as np
import pandas as pd

In [3]:
molecular_data = pd.read_csv('../use_data/rna_data.csv', index_col=0)
drugs = pd.read_csv('../use_data/embeddings.csv', index_col=0)
print(drugs.shape, molecular_data.shape)

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,H3C2,H3C3,AC098582.1,DUS4L-BCAP29,C8orf44-SGK3,ELOA3B,NPBWR1,ELOA3D,ELOA3,CDR1
ACH-001113,4.331992,0.000000,7.364660,2.792855,4.471187,0.028569,1.226509,3.044394,6.500005,4.739848,...,2.689299,0.189034,0.201634,2.130931,0.555816,0.000000,0.275007,0.0,0.000000,0.000000
ACH-001289,4.567424,0.584963,7.106641,2.543496,3.504620,0.000000,0.189034,3.813525,4.221877,3.481557,...,1.286881,1.049631,0.321928,1.464668,0.632268,0.000000,0.014355,0.0,0.000000,0.000000
ACH-001339,3.150560,0.000000,7.379118,2.333424,4.228049,0.056584,1.310340,6.687201,3.682573,3.273516,...,0.594549,1.097611,0.831877,2.946731,0.475085,0.000000,0.084064,0.0,0.000000,0.042644
ACH-001538,5.085340,0.000000,7.154211,2.545968,3.084064,0.000000,5.868390,6.165309,4.489928,3.956986,...,0.214125,0.632268,0.298658,1.641546,0.443607,0.000000,0.028569,0.0,0.000000,0.000000
ACH-000242,6.729417,0.000000,6.537917,2.456806,3.867896,0.799087,7.208478,5.570159,7.127117,4.568032,...,1.117695,2.358959,0.084064,1.910733,0.000000,0.000000,0.464668,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000285,0.056584,0.000000,6.604368,3.266037,4.973152,0.411426,0.097611,0.704872,4.829850,5.178715,...,2.229588,0.084064,1.310340,3.039138,0.344828,0.000000,0.000000,0.0,0.475085,0.042644
ACH-002669,3.111031,0.000000,7.031329,1.541019,3.664483,0.014355,3.624101,6.805421,4.472488,4.397118,...,0.189034,0.400538,0.356144,1.327687,0.000000,0.000000,0.014355,0.0,0.000000,0.000000
ACH-001858,4.390943,0.000000,7.013239,1.887525,3.252476,0.028569,3.286881,6.902194,5.410748,3.401903,...,1.097611,0.400538,0.613532,1.992768,0.704872,0.000000,1.464668,0.0,0.000000,0.526069
ACH-001997,5.057450,0.000000,7.815191,2.538538,3.893362,0.028569,4.079805,6.971659,4.469886,3.463361,...,0.831877,0.847997,1.292782,2.153805,0.687061,0.000000,0.000000,0.0,0.000000,0.000000


In [14]:
#read IC50 and merge?
auc_labels = pd.read_csv('../data/secondary-screen-dose-response-curve-parameters.csv')[['depmap_id', 'name', 'auc']]
auc_labels = auc_labels.rename(columns = {'depmap_id':'cell_line', 'name': 'DRUG'})
#IC50_labels = IC50_labels.merge(labels_filtered, how = 'inner')

auc_labels['DRUG'] = auc_labels['DRUG'].str.upper()
#auc_labels = auc_labels.merge(labels_filtered, how = 'inner')
print(auc_labels.shape)

auc_labels = auc_labels[['cell_line', 'DRUG', 'auc']].drop_duplicates()

auc_labels = auc_labels[~np.isnan(auc_labels['auc'])].reset_index(drop=True)
auc_labels = auc_labels[~np.isinf(auc_labels['auc'])].reset_index(drop=True)
print(auc_labels.shape)

auc_labels = auc_labels[np.isin(auc_labels['cell_line'], molecular_data.index)]
print(auc_labels.shape)

auc_labels = auc_labels[np.isin(auc_labels['DRUG'], drugs.columns)]
print(auc_labels.shape)

#
auc_labels['auc_per_drug'] = auc_labels.groupby('DRUG')['auc'].transform(lambda x: (x - x.mean()) / x.std())
auc_labels

/tmp/ipykernel_4176673/2538458575.py:2: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  auc_labels = pd.read_csv('../data/secondary-screen-dose-response-curve-parameters.csv')[['depmap_id', 'name', 'auc']]


(701004, 3)
(700921, 3)
(684170, 3)
(549552, 3)


,cell_line,DRUG,auc,auc_per_drug
0,ACH-000879,CYTARABINE,1.677789,2.667240
1,ACH-000320,CYTARABINE,1.240300,-0.168314
2,ACH-001145,CYTARABINE,1.472333,1.335587
3,ACH-000873,CYTARABINE,1.207160,-0.383109
4,ACH-000855,CYTARABINE,1.229332,-0.239405
...,...,...,...,...
700916,ACH-001321,BREQUINAR,0.987409,0.977227
700917,ACH-001321,AZD8931,1.000000,0.926736
700918,ACH-001321,AZD2014,1.000000,1.764364
700919,ACH-001321,MOTESANIB,0.888088,-0.419201


In [19]:
auc_labels['means'] = auc_labels.groupby('DRUG')['auc'].transform(lambda x: x.mean())
auc_labels['stds'] = auc_labels.groupby('DRUG')['auc'].transform(lambda x: x.std())
auc_labels['range'] = auc_labels.groupby('DRUG')['auc'].transform(lambda x: x.max()-x.min())

auc_labels

,cell_line,DRUG,auc,auc_per_drug,means,stds,range
0,ACH-000879,CYTARABINE,1.677789,2.667240,1.266269,0.154287,0.997791
1,ACH-000320,CYTARABINE,1.240300,-0.168314,1.266269,0.154287,0.997791
2,ACH-001145,CYTARABINE,1.472333,1.335587,1.266269,0.154287,0.997791
3,ACH-000873,CYTARABINE,1.207160,-0.383109,1.266269,0.154287,0.997791
4,ACH-000855,CYTARABINE,1.229332,-0.239405,1.266269,0.154287,0.997791
...,...,...,...,...,...,...,...
700916,ACH-001321,BREQUINAR,0.987409,0.977227,0.844464,0.146276,0.928873
700917,ACH-001321,AZD8931,1.000000,0.926736,0.838916,0.173818,1.407289
700918,ACH-001321,AZD2014,1.000000,1.764364,0.767408,0.131827,0.829653
700919,ACH-001321,MOTESANIB,0.888088,-0.419201,0.922472,0.082023,0.743482


In [24]:
auc_labels.to_csv('../use_data/prediction_targets.csv')
auc_labels

,cell_line,DRUG,auc,auc_per_drug,means,stds,range
0,ACH-000879,CYTARABINE,1.677789,2.667240,1.266269,0.154287,0.997791
1,ACH-000320,CYTARABINE,1.240300,-0.168314,1.266269,0.154287,0.997791
2,ACH-001145,CYTARABINE,1.472333,1.335587,1.266269,0.154287,0.997791
3,ACH-000873,CYTARABINE,1.207160,-0.383109,1.266269,0.154287,0.997791
4,ACH-000855,CYTARABINE,1.229332,-0.239405,1.266269,0.154287,0.997791
...,...,...,...,...,...,...,...
700916,ACH-001321,BREQUINAR,0.987409,0.977227,0.844464,0.146276,0.928873
700917,ACH-001321,AZD8931,1.000000,0.926736,0.838916,0.173818,1.407289
700918,ACH-001321,AZD2014,1.000000,1.764364,0.767408,0.131827,0.829653
700919,ACH-001321,MOTESANIB,0.888088,-0.419201,0.922472,0.082023,0.743482


In [25]:
check_duplicates = auc_labels[['cell_line', 'DRUG']]
check_duplicates.drop_duplicates()

,cell_line,DRUG
0,ACH-000879,CYTARABINE
1,ACH-000320,CYTARABINE
2,ACH-001145,CYTARABINE
3,ACH-000873,CYTARABINE
4,ACH-000855,CYTARABINE
...,...,...
700886,ACH-001321,CINACALCET
700896,ACH-001321,INC-280
700904,ACH-001321,DOVITINIB
700906,ACH-001321,LINSITINIB
